In [1]:
import pandas as pd 
import numpy as np
import DM_Utils as du
from datetime import datetime, timedelta
import glob
#import sed
from google.cloud import bigquery
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(r"C:\Users\55119\Documents\JHOE\fluent-optics-284222-a7285dcecd89.json")                                                                
client = bigquery.Client(credentials=credentials, project="fluent-optics-284222", location="US")


C:\Users\55119\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\55119\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\55119\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Caminho do arquivo no bucket
path = "gs://projeto-exemplo/QUOD/ENTRADA/Base_Modelagem"

In [3]:
Recebida = pd.read_csv(path, sep=',')
print ('A base pussui {} linhas e {} colunas'.format(Recebida.shape[0],Recebida.shape[1]))
Recebida.head()

C:\Users\55119\anaconda3\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


A base pussui 30000 linhas e 178 colunas


,SAFRA,RESPOSTA,VAR001,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,...,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,ID,VAR175
0,202303,0,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-308.0,-1000000,...,-1000000,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-1000000,178204,-1000000.0
1,202301,0,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-308.0,-1000000,...,-1000000,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-1000000,142317,-1000000.0
2,202301,0,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-308.0,-1000000,...,-1000000,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-1000000,340411,-1000000.0
3,202302,0,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-308.0,-1000000,...,-1000000,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-1000000,469,-1000000.0
4,202304,0,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-308.0,-1000000,...,-1000000,-1000000,-1000000,-1000000,-1000000.0,-1000000.0,-1000000.0,-1000000,164453,-1000000.0


In [4]:
#Cruzar os argumentos das 2 colunas e somar as variáveis
pd.crosstab(Recebida['RESPOSTA'],Recebida['SAFRA'])

SAFRA,202301,202302,202303,202304,202305,202306
RESPOSTA,,,,,,
0,5041,4352,4880,4296,4812,4083
1,454,438,454,385,432,373


In [5]:
cols_in = Recebida.columns.tolist()

In [6]:
#Separando em dados de treinamento e teste
#------------------------------------------------------------
y = Recebida['RESPOSTA']
X = Recebida[cols_in] # Com ALVO temporariamente
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 125)


In [7]:
#Selecionando Atributos - RFE Recursive Feature Elimination
#---------------------------------------------------------------------------
#Feature extraction
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

model = LogisticRegression(solver='newton-cg')
selected = RFE(model,step=15,n_features_to_select=35).fit(X_train, y_train)

print()
print('----------------     SELEÇÃO DE VARIÁVEIS--------------------------------------')
print("Num Features: %d" % selected.n_features_)
used_cols = []
for i in range(0, len(selected.support_)):
    if selected.support_[i]: 
        used_cols.append(X_train.columns[i]) 
        print('             -> {:30}     '.format(X_train.columns[i]))
print('-------------------------------------------------------------------------------')

X_train = X_train[used_cols]     # Carrega colunas de entrada selecionadas por RFE
X_test = X_test[used_cols]       # Carrega colunas de entrada selecionadas por RFE

C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to conver


----------------     SELEÇÃO DE VARIÁVEIS--------------------------------------
Num Features: 35
             -> RESPOSTA                           
             -> VAR007                             
             -> VAR009                             
             -> VAR016                             
             -> VAR025                             
             -> VAR029                             
             -> VAR035                             
             -> VAR037                             
             -> VAR040                             
             -> VAR045                             
             -> VAR047                             
             -> VAR054                             
             -> VAR065                             
             -> VAR071                             
             -> VAR072                             
             -> VAR084                             
             -> VAR086                             
             -> VA

C:\Users\55119\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


In [8]:
#Regressão linear com dados de treinamento
from sklearn.linear_model import LinearRegression
LinearReg = LinearRegression(fit_intercept=True)
LinearReg.fit(X_train, y_train)

LinearRegression()

In [9]:
#Rede Neural com dados de treinamento
from sklearn.neural_network import MLPClassifier 
RNA = MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=True,
       epsilon=1e-08, hidden_layer_sizes=(25), learning_rate='constant',
       learning_rate_init=0.01, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.25, verbose=False,
       warm_start=False)
RNA.fit(X_train, y_train)

C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\55119\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid featur

MLPClassifier(activation='logistic', alpha=1e-05, early_stopping=True,
              hidden_layer_sizes=25, learning_rate_init=0.01, max_iter=2000,
              random_state=1, validation_fraction=0.25)

In [10]:
#Previsão treinamento e teste - CLASSIFICAÇÃO
#---------------------------------------------------------------------------

#Regressão Linear
y_pred_train_RL = np.array([1 if x > 0.5 else 0 for x in LinearReg.predict(X_train)] )
y_pred_test_RL  = np.array([1 if x > 0.5 else 0 for x in LinearReg.predict(X_test)])
#Redes Neurais
y_pred_train_RNA = RNA.predict(X_train)
y_pred_test_RNA = RNA.predict(X_test)

In [12]:
#Calcula e mostra a Acurácia dos modelos
#---------------------------------------------------------------------------


from sklearn import metrics
print()
print('-----------------------------   ACURÁCIA   ---------------------------------')

print('Acurácia Regressão Linear:    ',metrics.accuracy_score(y_test, y_pred_test_RL))
print('Acurácia Redes Neurais:       ',metrics.accuracy_score(y_test, y_pred_test_RNA))
print('----------------------------------------------------------------------------')
print()


-----------------------------   ACURÁCIA   ---------------------------------
Acurácia Regressão Linear:     1.0
Acurácia Redes Neurais:        0.9144
----------------------------------------------------------------------------

